**Nvidia kart kontrolü**

In [ ]:
!nvidia-smi

Sat May  8 10:45:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Google colabda çalıştığı için eğitim ile çalışmaları drive üstünde yapması için bağlanmakta

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
get_ipython().system('ln -s /content/gdrive/My\\ Drive/ /mydrive')
get_ipython().system('ls /mydrive')

Mounted at /content/gdrive
'Colab Notebooks'   yoloo


**1) Darknetin indirilmesi**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14997, done.
remote: Total 14997 (delta 0), reused 0 (delta 0), pack-reused 14997
Receiving objects: 100% (14997/14997), 13.38 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (10194/10194), done.


**2) Darknetin nvidia gpu, cuda ve opencv açık versiyonu oluşturulur**


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Darknet ağının yolov3 eğitime uygun olan ayar dosyası alarak eğitilmesi için kopyalar**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

4) yolov3 ayar dosyası üzerinde eğitim ayarlarının yapılması
- batch sayısı 64
- subdivision sayısı 16
- class ve filtre sayıları tek sınıf için ayarlandı


In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**drive üstünde ağırlıkların kayıt edileceği dosyayı oluştur**

In [ ]:

!mkdir "/mydrive/yolov3"

**obje dosyasına sınıf isminin yazılması**

In [ ]:
!echo "Fire" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

**eğitmek için kullanılan darknt53 ağının indirilmesi**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-05-08 10:49:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  97.4MB/s    in 1.6s    

2021-05-08 10:49:17 (97.4 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**drive içinde zip olarak atılmış olan fotoğrafların çıkarılması**

In [ ]:
!unzip /mydrive/yoloo/fire-detect.v3i.darknet.zip -d data/obj

Archive:  /mydrive/yoloo/fire-detect.v3i.darknet.zip
   creating: data/obj/fire-detect.v2i.darknet/
  inflating: data/obj/fire-detect.v2i.darknet/fire-101_png.rf.0bbb5e42931847b5fca8c9701f0b767b.jpg  
  inflating: data/obj/fire-detect.v2i.darknet/fire-101_png.rf.0bbb5e42931847b5fca8c9701f0b767b.txt  
  inflating: data/obj/fire-detect.v2i.darknet/fire-101_png.rf.f3f35d8d27e5802352b9b2db940933c6.jpg  
  inflating: data/obj/fire-detect.v2i.darknet/fire-102_png.rf.0ba53fb61f3daf8016014d50f8b39f4e.jpg  
  inflating: data/obj/fire-detect.v2i.darknet/fire-102_png.rf.6042e29119c6abf9e2ea441a97450600.jpg  
  inflating: data/obj/fire-detect.v2i.darknet/fire-102_png.rf.6042e29119c6abf9e2ea441a97450600.txt  
  inflating: data/obj/fire-detect.v2i.darknet/fire-105_png.rf.9de36b41e0c5a72f938db1253e0f6d1d.txt  
  inflating: data/obj/fire-detect.v2i.darknet/fire-106_png.rf.05babb165cfc5c5954e560950347e1b6.txt  
  inflating: data/obj/fire-detect.v2i.darknet/fire-106_png.rf.2e7ae0246deffcc74174cd5f1cda14

**text dosyası içerisindeki**

In [ ]:

import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/fire-detect.v2i.darknet/*.txt")
for i, file_path in enumerate(txt_file_paths):
    
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            sayilar = re.findall("[0-9.]+", line)
            print(sayilar)
            if sayilar:

                
                text = "{} {} {} {} {}".format(0, sayilar[1], sayilar[2], sayilar[3], sayilar[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
['1', '0.442', '0.72', '0.244', '0.348']
545 data/obj/fire-detect.v2i.darknet/fire-32_png.rf.21e69de2b6a8e2893453a89921d56020.txt
0 0.442 0.72 0.244 0.348
1 0.592 0.568 0.58 0.212

['1', '0.592', '0.568', '0.58', '0.212']
546 data/obj/fire-detect.v2i.darknet/fire-321_png.rf.66a644aaf89d6d32426eb94574f0da5a.txt
0 0.592 0.568 0.58 0.212
1 0.892 0.516 0.164 0.14
['1', '0.892', '0.516', '0.164', '0.14']
546 data/obj/fire-detect.v2i.darknet/fire-321_png.rf.66a644aaf89d6d32426eb94574f0da5a.txt
0 0.892 0.516 0.164 0.14
1 0.508 0.432 0.456 0.308
['1', '0.508', '0.432', '0.456', '0.308']
547 data/obj/fire-detect.v2i.darknet/fire-245_png.rf.28d54e52d6ae01d62b109fbaca623ba4.txt
0 0.508 0.432 0.456 0.308
1 0.588 0.368 0.324 0.464
['1', '0.588', '0.368', '0.324', '0.464']
548 data/obj/fire-detect.v2i.darknet/fire-256_png.rf.d99f4c92fbf117e1f8fc959fa33d8646.txt
0 0.588 0.368 0.324 0.464
1 0.296 0.446 0.292 0.24

['1', '0.296', '0.446', '0.292', '0.2

**Jpg dosyalarının listelenmesi**

In [ ]:
import glob
images_list = glob.glob("data/obj/fire-detect.v2i.darknet/*.jpg")
print(images_list)

['data/obj/fire-detect.v2i.darknet/fire-226_png.rf.762932489d349d705cf0e4e6b3306341.jpg', 'data/obj/fire-detect.v2i.darknet/fire-68_png.rf.ed8cffef271fa896f0e8255ca486ed84.jpg', 'data/obj/fire-detect.v2i.darknet/fire-69_png.rf.edb0d2e4df46961d567aecface492e50.jpg', 'data/obj/fire-detect.v2i.darknet/fire-558_png.rf.8252eb8590f9b463be3e76e0d40a5fdb.jpg', 'data/obj/fire-detect.v2i.darknet/fire-320_png.rf.20ac0103c55cdc7b908d1aec6b760a38.jpg', 'data/obj/fire-detect.v2i.darknet/fire-705_png.rf.000565c3f09eca7ceee3e340ba1954d1.jpg', 'data/obj/fire-detect.v2i.darknet/fire-130_png.rf.16d311d9a01ae78b2bbd52650b00cc9e.jpg', 'data/obj/fire-detect.v2i.darknet/fire-745_png.rf.4f38789b50865233a41ef2ef06da4c2d.jpg', 'data/obj/fire-detect.v2i.darknet/fire-613_png.rf.b3ecad98cf26e0cb1639ff1f78a9dc1b.jpg', 'data/obj/fire-detect.v2i.darknet/fire-320_png.rf.4f12b02a3ecb29aed820591425a5499b.jpg', 'data/obj/fire-detect.v2i.darknet/fire-34_png.rf.38b8ab86915f2b42ab9163cd032683fb.jpg', 'data/obj/fire-detect.v

**eğitim için dosyaların uzantılarının yazılamsı örnek: Desktop/test/test1.jpg**

In [ ]:
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) eğitime başla (google colab 1-2 saatte bi uyarı veriyo onu  kontrol et bide ücretsizde 12 saat max limit var) map fonksiyonu ile mean average precision buluna biliyor ve >/dosya yolu/text.log ile sistem log dosyası yazdırılır ve burdan sistem doğruluğu alınabilir**


In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 > /content/gdrive/MyDrive/yolov3/train.log -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000141, iou_loss = 0.000000, total_loss = 0.000141 
 total_bbox = 72368, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.579324), count: 6, class_loss = 1.409497, iou_loss = 1.048800, total_loss = 2.458297 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.526586), count: 1, class_loss = 0.250408, iou_loss = 0.290216, total_loss = 0.540623 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000227, iou_loss = 0.000000, total_loss = 0.000227 
 total_bbox = 72375, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.562092), count: 5, class_loss = 1.166020, iou_loss = 1.192137, total_loss = 2.358156 
v3 (mse